# Real Time Processing - Part 1

## Connecting to AWS Via Kinesis and streaming tweets through the Twitter API

This notebook provides the necessary code to connect the Twitter Stream API to an AWS kinesis app

To being with, we need to have:
   - Your AWS Account Set up and AWS CLI installed and configured to your account
   - Created an app on twitter developer, and have the below-mentioned credentials
   - The boto3, TwitterAPI and tweepy libraries installed for Python

Reading in Twitter Credentials

In [1]:
#Reading in twitter credentials
consumer_key = "XXXXXXXXXXXXXXXX"
consumer_secret = "XXXXXXXXXXXXXXXXXXX"
access_token_key = "XXXXXXXXXXXXXXx-XXXXXXXXXXXXXXXXXXXX"
access_token_secret = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

Creating a Kinesis Stream using the boto3 library. The stream will automatically created with the below streamname in the AWS console

In [25]:
#creating the kinesis stream
import boto3

client = boto3.client('kinesis')
response = client.create_stream(
   StreamName='twitter_bigdata', #your streamname here
   ShardCount=1
)

Connecting the API to kinesis streams. The kinesis streams app should already be set up in the AWS console. The data will be streamed in a JSON format through the Twitter API

The API filter can also be set. The commented out options have been tested

In [ ]:
#importing the necessary packages
from TwitterAPI import TwitterAPI
import json
import boto3
#import twitterCreds


#accessing the API
api = TwitterAPI(consumer_key, consumer_secret, access_token_key, access_token_secret)

kinesis = boto3.client('kinesis')

r = api.request('statuses/filter', {'follow':'629747990'})

#for locations
#r = api.request('statuses/filter', {'locations':'-90,-90,90,90'})
#for userids @abcdef:
#r = api.request('statuses/filter', {'follow':'123456'})
#for general text searches
#r = api.request('statuses/filter', {'track':'iphone'})



for item in r:
    kinesis.put_record(StreamName="twitter_bigdata", Data=json.dumps(item), PartitionKey="filler")

A script to test the Kinesis Stream output.The json outputs will be displayed below

In [ ]:
import boto3
import time
import json
## aws creds are stored in ~/.boto
kinesis = boto3.client("kinesis")
shard_id = "shardId-000000000000" #only one shard!
pre_shard_it = kinesis.get_shard_iterator(StreamName="twitter", ShardId=shard_id, ShardIteratorType="LATEST")
shard_it = pre_shard_it["ShardIterator"]
while 1==1:
     out = kinesis.get_records(ShardIterator=shard_it, Limit=1)
     shard_it = out["NextShardIterator"]
     print out;
     time.sleep(1.0)